<a href="https://colab.research.google.com/github/sanjaynagi/Ano-express/blob/main/workflow/notebooks/plot-gene-expression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anoexpress -q

![image](https://raw.githubusercontent.com/sanjaynagi/ano-expressir/main/docs/logo.png) 


In [1]:
import anoexpress as xpress

In [2]:
#@title **Ano-express** { run: "auto" }   
#@markdown This notebook produces interactive strip and boxplots with plotly, to summarise gene expression across 35 *Anopheles* insecticide resistant v susceptible RNA-Sequencing experiments. There are four different analyses to select
#@markdown 'gamb_colu', 'gamb_colu_arab', 'gamb_colu_arab_fun' and 'fun'.  
#@markdown each which integrates a different combination of Anopheles species – ‘gamb_colu’ (An. gambiae and An. coluzzii), ‘gamb_colu_arab’ (adding An. arabiensis), ‘gamb_colu_arab_fun’ (adding An. funestus), and finally ‘fun’ (An. funestus alone).  
#@markdown As you include more species, there are less genes overall in the dataset, as we can only include genes with one-to-many orthologs between species.
#@markdown   
#@markdown Requesting feedback and ideas for how to explore the data.

analysis = "gamb_colu_arab_fun" #@param ['gamb_colu', 'gamb_colu_arab', 'gamb_colu_arab_fun', 'fun']
GeneID = "AGAP002865" #@param {type:"string"} 
plot_type = "strip" #@param ['strip', 'boxplot']
include_microarray_data = True #@param {type:'boolean'}

xpress.plot_gene_expression(
    analysis=analysis,
    gene_id=GeneID, 
    microarray=include_microarray_data,
    title="", 
    plot_type=plot_type, 
    height=300)

We can provide a list of genes, and sort by either AGAP identifier, or by the median fold-change. 

In [4]:
microarray=False
gene_id = ['AGAP002865', 'AGAP006227', 'AGAP006222', 'AGAP006228']
analysis = "gamb_colu_arab_fun"
sort_by="mean"

In [5]:
import numpy as np

In [22]:
# load the metadata and subset to the species of interest
df_metadata = xpress.metadata(analysis=analysis, microarray=microarray)
df_samples = xpress.sample_metadata(analysis=analysis)
# load count data, make long format and merge with metadata for hovertext
count_data = xpress.data(data_type="log2counts", analysis=analysis, microarray=microarray, gene_id=gene_id, sort_by=sort_by)
count_data = count_data.melt(id_vars='GeneID', var_name='sampleID', value_name='log2_counts')
count_data = count_data.merge(df_samples, how='left').assign(counts = lambda x: np.round(2**x.log2_counts, 0))

In [6]:
count_data = xpress.data(data_type="log2counts", analysis=analysis, microarray=microarray, gene_id=gene_id, sort_by=sort_by)

In [7]:
count_data

,GeneID,Tie1,Tie2,Tie3,Ban1,Ban2,Ban3,Ban4,Bak1,Bak2,...,Malawi_fun10,Uganda_fun1,Uganda_fun2,Uganda_fun3,Uganda_fun4,Uganda_fun5,Uganda_fun6,Uganda_fun7,Uganda_fun8,Uganda_fun9
1,AGAP006222,6.44,5.45,6.03,7.45,8.50,7.79,7.59,7.64,7.50,...,8.67,6.71,7.54,7.55,6.93,8.92,7.37,8.36,7.99,8.28
2,AGAP006227,8.25,7.88,7.97,7.31,7.74,7.37,7.70,7.85,7.40,...,7.21,6.25,6.62,6.70,6.64,7.53,6.70,7.18,7.17,7.16
0,AGAP002865,8.30,7.94,8.25,8.13,9.25,8.70,8.40,8.19,8.39,...,12.37,8.74,9.03,9.00,9.10,10.92,9.67,9.55,9.30,9.89
3,AGAP006228,9.26,9.34,9.06,8.49,8.81,8.41,8.51,9.02,9.28,...,8.74,7.94,7.98,8.31,8.36,9.01,8.19,8.74,8.69,8.48


In [25]:
fc_data = xpress.data(data_type="fcs", analysis=analysis, microarray=microarray, gene_id=gene_id, sort_by=sort_by)

In [28]:
count_data.set_index('GeneID').loc[fc_data['GeneID']]

,Tie1,Tie2,Tie3,Ban1,Ban2,Ban3,Ban4,Bak1,Bak2,Bak3,...,Malawi_fun10,Uganda_fun1,Uganda_fun2,Uganda_fun3,Uganda_fun4,Uganda_fun5,Uganda_fun6,Uganda_fun7,Uganda_fun8,Uganda_fun9
GeneID,,,,,,,,,,,,,,,,,,,,,
AGAP006228,9.26,9.34,9.06,8.49,8.81,8.41,8.51,9.02,9.28,9.06,...,8.74,7.94,7.98,8.31,8.36,9.01,8.19,8.74,8.69,8.48
AGAP006227,8.25,7.88,7.97,7.31,7.74,7.37,7.70,7.85,7.40,8.96,...,7.21,6.25,6.62,6.70,6.64,7.53,6.70,7.18,7.17,7.16
AGAP006222,6.44,5.45,6.03,7.45,8.50,7.79,7.59,7.64,7.50,7.86,...,8.67,6.71,7.54,7.55,6.93,8.92,7.37,8.36,7.99,8.28
AGAP002865,8.30,7.94,8.25,8.13,9.25,8.70,8.40,8.19,8.39,8.88,...,12.37,8.74,9.03,9.00,9.10,10.92,9.67,9.55,9.30,9.89


In [26]:
fc_data

,GeneID,GeneName,Tiefora_v_Ngousso,Gou_v_Moz,Ban_v_BanS,BanRe_v_BanS,Bak_v_Kisumu,VK7_v_Kisumu,Cameroon_v_Ngousso,Chad_v_Ngousso,...,Ghana_fun_v_Fang,Malawi_fun_v_Fang,Uganda_fun_v_Fang,Asendabo_v_Moz,Chewaka_v_Moz,Tolay_v_Moz,Ethiopia_v_Dongola,Bouake_gamb_unexp_v_Kisumu,Bouake_colu_unexp_v_Ngousso,BusiaSurvivors_v_Kisumu
1,AGAP006228,COEAE2F,-0.44,-0.12,-0.41,-0.11,-0.26,-0.30,-0.20,0.24,...,0.36,0.28,0.18,0.10,0.36,0.22,0.18,-0.10,-0.73,0.75
2,AGAP006227,NaN,-0.46,0.01,-0.98,-0.37,0.26,0.01,-0.41,-0.17,...,0.04,-0.36,-0.48,-0.28,0.01,-0.08,0.11,2.57,0.66,0.59
3,AGAP006222,NaN,0.75,1.27,2.17,0.03,1.53,1.70,1.73,1.10,...,0.71,0.29,-0.03,2.22,3.44,2.52,1.08,0.15,2.43,-2.83
0,AGAP002865,CYP6P3,4.26,1.07,1.36,-0.76,2.03,3.90,1.26,1.87,...,3.58,5.44,2.61,2.41,4.70,2.85,0.47,1.00,5.19,-3.39


In [ ]:

# load fold change data, make long format and merge with metadata for hovertext
fc_data = xpress.data(data_type="fcs", analysis=analysis, microarray=microarray, gene_id=gene_id, sort_by=sort_by).reset_index()
count_data = count_data.query("GeneID in @fc_data.GeneID")
fc_data.loc[:, 'Label'] = [id_ + " | " + name if name != "" else id_ for id_, name in zip(fc_data['GeneID'].fillna(""), fc_data['GeneName'].fillna(""))]
fc_data = fc_data.drop(columns=['GeneName', 'GeneID']).melt(id_vars='Label', var_name='comparison', value_name='log2FC')
fc_data = fc_data.merge(df_metadata, how='left')

In [5]:
xpress.plot_gene_expression(
    analysis="gamb_colu_arab_fun", 
    gene_id=["AGAP006222", "AGAP006227", "AGAP006228"], 
    microarray=False,
    sort_by=None,
    title="Coeae1f", 
    plot_type='strip', 
    height=300
)

You can also produce a boxplot, although the hovertext doesnt quite work as expected.

In [ ]:
xpress.plot_gene_expression(
    analysis='gamb_colu_arab_fun', 
    gene_id=["AGAP006222", "AGAP006227", "AGAP006228"], 
    sort_by='median', 
    plot_type='boxplot', 
    height=300
    )